### Agents 概念

智能体将语言模型与工具相结合，创建能够对任务进行推理、决定使用哪些工具并迭代地寻求解决方案的系统。(ReAct)
create_agent 提供了一个可用于生产环境的代理实现。
LLM 智能体循环运行各种工具以实现目标 。智能体持续运行，直到满足停止条件为止——即模型发出最终输出或达到迭代次数限制。
如下图

<div align=center><img src="images/image.png" width=20%></div>

### 1 核心组件

#### 1.1 模型 Models

模型是智能体的推理引擎。它可以通过多种方式进行指定，支持静态和动态模型选择。

##### 1.1.1 静态模型

静态模型在创建代理时配置一次，并在整个执行过程中保持不变。这是最常见、最直接的方法。

In [1]:
from langchain.agents import create_agent

# 这是直接把模型放入Agent中
agent = create_agent(
    "deepseek-chat"
)

In [2]:
# 为了更好控制模型配置，自定义初始化模型。
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.6,
    max_tokens=1000,
    timeout=30,
    #...
)
agent = create_agent(model)

##### 1.1.2 动态模型

动态模型根据当前状态和上下文在运行时进行选择。

要使用动态模型，使用 @wrap_model_call 装饰器创建中间件，该装饰器会在请求中修改模型

In [3]:
from langchain_deepseek import ChatDeepSeek
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatDeepSeek(model="deepseek-chat")
advanced_model = ChatDeepSeek(model="deepseek-reasoner")
# @wrap_model_call是一个环绕式中间件装饰器，它允许您在模型调用前后拦截和修改请求/响应。这是LangChain中间件系统中最强大的钩子之一。可以决定是否调用、何时调用、调用多少次处理器， 接收ModelRequest和处理器函数，返回ModelResponse    
# ModelRequest 作用：封装模型调用请求，包含所有发送给模型的信息。
# ModelResponse 作用：封装模型调用响应，包含模型返回的结果。
# handler  调用handler = 继续执行，不调用 = 中断执行

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """根据对话复杂度选择模型"""
    message_count = len(request.state["messages"])

    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model
    request.model = model
    return handler(request)

agent = create_agent(
    model=basic_model,
    middleware=[dynamic_model_selection]
)
# 使用结构化输出时，不支持预绑定模型（已调用 bind_tools 的模型）。如果需要进行动态模型选择并使用结构化输出，请确保传递给中间件的模型不是预绑定的。

#### 1.2 工具 Tools

工具赋予代理执行操作的能力。代理不仅仅是简单的模型绑定工具，还能实现：多次工具调用（由单个提示触发），在适当情况下的并行工具调用，根据之前的结果动态选择工具，工具重试逻辑与错误处理，在工具调用之间保持状态。

##### 1.2.1 定义工具

In [4]:
from langchain.tools import tool
from langchain.agents import create_agent

@tool
def search(query):
    """输入query，用来搜索内容"""
    return f"{query}搜索到了三个城市。分别是上海，北京，深圳。"

@tool
def get_weather(city):
    """根据city来得到所在city的天气"""
    return f"{city}的天气是未知的。"

agent = create_agent(
    model = model,
    tools = [search, get_weather]
)

##### 1.2.2 工具错误处理

要自定义工具错误的处理方式，请使用 @wrap_tool_call 装饰器创建中间件，类似上面的@wrap_model_call

In [5]:
from langchain.agents.middleware import wrap_tool_call
from langchain.agents import create_agent
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """使用自定义消息处理工具执行错误。"""
    try:
        return handler(request)
    except Exception as e:
        # 向模型返回自定义错误消息
        return ToolMessage(
            content=f"工具错误：请检查您的输入并重试。({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model = model,
    tools = [search, get_weather],
    middleware=[handle_tool_errors]
)

In [28]:
"""
当工具失败时，代理将返回带有自定义错误信息的 ToolMessage ：
[
    ...
    ToolMessage(
        content="Tool error: Please check your input and try again. (division by zero)",
        tool_call_id="..."
    ),
    ...
]
"""

'\n当工具失败时，代理将返回带有自定义错误信息的 ToolMessage ：\n[\n    ...\n    ToolMessage(\n        content="Tool error: Please check your input and try again. (division by zero)",\n        tool_call_id="..."\n    ),\n    ...\n]\n'

##### 1.2.3 在 ReAct 循环中的工具使用

代理遵循 ReAct（“推理 + 行动”）模式，在简短的推理步骤与有针对性的工具调用之间交替进行，并将所得观察结果输入到后续决策中，直到能够给出最终答案。

In [6]:
# 简单示例，虽然不怎么好。
from langchain.messages import HumanMessage
for chunk in agent.stream(
    {"messages": [HumanMessage(content="你使用search工具进行搜索，搜索魔法，把搜索得到的三个城市，分别帮我查一下天气，再对每个城市的天气分别在进行使用search搜索")]},
    stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

你使用search工具进行搜索，搜索魔法，把搜索得到的三个城市，分别帮我查一下天气，再对每个城市的天气分别在进行使用search搜索
================================== Ai Message ==================================

我来帮您搜索"魔法"相关内容，然后对搜索到的三个城市分别查询天气并进行进一步搜索。
Tool Calls:
  search (call_00_oqfhXnthYUVZdAP53WR7TdZh)
 Call ID: call_00_oqfhXnthYUVZdAP53WR7TdZh
  Args:
    query: 魔法
================================= Tool Message =================================
Name: search

魔法搜索到了三个城市。分别是上海，北京，深圳。
================================== Ai Message ==================================

现在我来为这三个城市分别查询天气：
Tool Calls:
  get_weather (call_00_nZHBdH3JUCl6b7fJu493AA9j)
 Call ID: call_00_nZHBdH3JUCl6b7fJu493AA9j
  Args:
    city: 上海
  get_weather (call_01_vrCLLHRGLsa6C63kxAGiqAOF)
 Call ID: call_01_vrCLLHRGLsa6C63kxAGiqAOF
  Args:
    city: 北京
  get_weather (call_02_qbbZqhk3dXCf2WJ1TjN5yalS)
 Call ID: call_02_qbbZqhk3dXCf2WJ1TjN5yalS
  Args:
    city: 深圳
================================= To

#### 1.3 系统提示 System Prompt

你可以通过提供提示来塑造你的代理处理任务的方式。 system_prompt 参数可以作为字符串提供.

当未提供 system_prompt 时，代理将直接从消息中推断其任务。

In [7]:
agent = create_agent(
    model,
    tools = [search, get_weather],
    system_prompt="你是一个乐于助人的助手。请简洁准确的回答问题。"
)

##### 1.3.1 动态系统提示 Dynamic System prompt

对于需要根据运行时上下文或代理状态修改系统提示的更高级用例，可以使用中间件。

@dynamic_prompt 装饰器创建了根据模型请求动态生成系统提示的中间件：专门用于生成动态系统提示，相当于 @wrap_model_call 装饰器的简化版本，专门用于修改提示，属于"包装式"中间件，在模型调用前后执行。

In [8]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """根据用户角色生成系统提示"""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "你是一个优秀的AI助手。"

    if user_role == "expert":
        return f"{base_prompt} 提供详细的 technical 回应."
    elif user_role == "beginner":
        return f"{base_prompt} 用简单的方式解释概念，避免使用专业术语."

    return base_prompt

agent = create_agent(
    model="deepseek-chat",
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context
)

# 系统提示将根据上下文动态设置
result = agent.invoke(
    {"messages": [{"role": "user", "content": "解释机器学习"}]},
    context={"user_role": "expert"}
)

#### 1.4 结构化输出

在某些情况下，您可能希望代理返回特定格式的输出。LangChain 通过 response_format 参数提供了结构化输出的策略。

ToolStrategy(工具策略) 使用人工工具调用生成结构化输出。这适用于任何支持工具调用的模型

In [9]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="deepseek-chat",
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "我叫栗子，email是131515@sdf，phone为10086"}]
})

result

{'messages': [HumanMessage(content='我叫栗子，email是131515@sdf，phone为10086', additional_kwargs={}, response_metadata={}, id='ea23d012-63e3-4035-872a-2c96bbfef987'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 179, 'total_tokens': 208, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 51}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '9af6f79a-1a0a-4dec-a80b-afebfa8a1dc4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e1708958-1edd-4823-920d-f8d903ec9146-0', tool_calls=[{'name': 'ContactInfo', 'args': {'name': '栗子', 'email': '131515@sdf', 'phone': '10086'}, 'id': 'call_00_zTg40BQ3xK8K98x1VZumybb9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 179, 'output_tokens': 29, 'total_toke

In [10]:
result["structured_response"]

ContactInfo(name='栗子', email='131515@sdf', phone='10086')

ProviderStrategy（提供者策略） 使用模型提供商的原生结构化输出生成。这更可靠，但仅适用于支持原生结构化输出的提供商

In [ ]:
from langchain.agents.structured_output import ProviderStrategy
from langchain.chat_models import init_chat_model


agent = create_agent(
    model="deepseek-chat",
    response_format=ProviderStrategy(ContactInfo)
)
result = agent.invoke({
    "messages": [{"role": "user", "content": "我叫栗子，email是131515@sdf，phone为10086"}]
})

"""
截至 langchain 1.0 ，仅仅传递架构（例如 response_format=ContactInfo ）已不再支持。您必须明确使用 ToolStrategy 或 ProviderStrategy 。
"""
result

{'messages': [HumanMessage(content='我叫栗子，email是131515@sdf，phone为10086', additional_kwargs={}, response_metadata={}, id='8e5f5299-606d-4f7c-bef6-db403afb50a8'),
  AIMessage(content='{"email":"131515@sdf","name":"栗子","phone":"10086"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 79, 'total_tokens': 111, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'openai/gpt-5', 'system_fingerprint': None, 'id': '20251113104111942069640IadDhWuw', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--7b336f97-0a1d-4df1-bf7e-166267bed736-0', usage_metadata={'input_tokens': 79, 'output_tokens': 32, 'total_tokens': 111, 'input_token_details': {}, 'output_token_details': {}})],
 'structured_response': ContactInfo(name='栗子', email='131515@sdf', phone='10086')}

#### 1.5 内存 Memory

Agents通过消息状态自动维护对话历史。还可以配置agent使用自定义状态架构，以在对话中记住额外的信息。 

存储在状态中的信息可以被视为代理的短期记忆

自定义状态架构必须继承 AgentState 作为 TypedDict 。

定义自定义状态有两种方式:  1 通过中间件（推荐）.   2 通过 state_schema 在 create_agent 上




##### 1.5.1 通过中间件定义状态

如果状态只在某个中间件体系中用，就把状态放到那个中间件里，而不是放到全局。

In [ ]:
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware

class CustomState(AgentState):
    user_prefer: dict

class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState

    def before_model(self, state: CustomState, runtime):
        
        ...
agent = create_agent(
    model,
    middleware=[CustomMiddleware()]
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "我更倾向于技术性解释"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

##### 1.5.2 通过 state_schema 定义状态

使用 state_schema 参数作为快捷方式，定义仅在工具中使用的自定义状态。

In [ ]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict

agent = create_agent(
    model,
    tools=[],
    state_schema=CustomState
)
result = agent.invoke({
    "messages": [{"role": "user", "content": "我更倾向于技术性解释"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})